In [1]:
from pyMSOO.MFEA.model import MaTGA
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 

In [38]:
import numpy as np
import random
from functools import reduce
import time

from pyMSOO.MFEA.model import AbstractModel
from pyMSOO.utils import Crossover, Mutation, Selection
from pyMSOO.utils.EA import *
from pyMSOO.utils.numba_utils import numba_randomchoice

class model(AbstractModel.model):
    def compile(self, 
        IndClass: Type[Individual],
        tasks: List[AbstractTask], 
        crossover: Crossover.SBX_Crossover, mutation: Mutation.PolynomialMutation, selection: Selection.ElitismSelection, 
        *args, **kwargs):
        super().compile(IndClass, tasks, crossover, mutation, selection, *args, **kwargs)
    
    def fit(self, nb_generations, rmp = 0.3, nb_inds_each_task = 100, evaluate_initial_skillFactor = True, *args, **kwargs) -> List[Individual]:
        super().fit(*args, **kwargs)

        # initialize population
        population = Population(
            self.IndClass,
            nb_inds_tasks = [nb_inds_each_task] * len(self.tasks), 
            dim = self.dim_uss,
            list_tasks= self.tasks,
            evaluate_initial_skillFactor = evaluate_initial_skillFactor
        )

        self.R_o = np.zeros((len(self.tasks))) 
        self.R_s = np.zeros((len(self.tasks)))
        self.E_o = np.zeros((len(self.tasks)))
        self.E_s = np.zeros((len(self.tasks)))

        # save history
        self.history_cost.append([ind.fcost for ind in population.get_solves()])
        
        self.render_process(0, ['Cost'], [self.history_cost[-1]], use_sys= True)

        for epoch in range(nb_generations):
            offsprings = Population(
                self.IndClass,
                nb_inds_tasks = [0] * len(self.tasks), 
                dim = self.dim_uss,
                list_tasks= self.tasks,
            )

            lambda_i = (self.R_o/(self.E_o + 1e-6))/(self.R_s/(self.E_s + 1e-6) + self.R_o/(self.E_o + 1e-6) + 1e-6)
            idx_other = np.where(np.random.rand(len(self.tasks)) < lambda_i)[0]
            idx_same = list(set(range(len(self.tasks))) - set(idx_other))

            self.E_o[idx_other] += nb_inds_each_task
            self.E_s[idx_same] += nb_inds_each_task
            
            # start = time.time()

            for i in idx_other:
                # self.E_o[i] += nb_inds_each_task
                offs = population.__getRandomInds__(nb_inds_each_task)
                res = np.fromiter(map(self.tasks[i].__call__, offs), float)
                self.R_o[i] += sum(res < population[i].__getBestIndividual__.fcost)
                for o in offs:
                    offsprings.__addIndividual__(o)
            
            # end = time.time()
            # print("A: ", end - start)

            # start = time.time()
            # print(len(idx_other))
            for i in idx_same:
                # current_best = population[i].__getBestIndividual__.fcost
                # self.E_s[i] += nb_inds_each_task
                offs = []
                
                while len(offs) < nb_inds_each_task:
                    # choose parent 
                    pa, pb = population[i].__getRandomItems__(2)
                    # intra / inter crossover
                    oa, ob = self.crossover(pa, pb, i, i)
                    # mutate
                    oa = self.mutation(oa, return_newInd= True)
                    oa.skill_factor = pa.skill_factor

                    ob = self.mutation(ob, return_newInd= True)    
                    ob.skill_factor = pb.skill_factor
                    
                    offsprings.__addIndividual__(oa)
                    offsprings.__addIndividual__(ob)

                    # self.R_s[i] += self.tasks[i](oa) > current_best
                    # self.R_s[i] += self.tasks[i](ob) > current_best

                    offs.append(oa)
                    offs.append(ob)

                res = np.fromiter(map(self.tasks[i].__call__, offs), float)
                self.R_s[i] += sum(res < population[i].__getBestIndividual__.fcost)      
            
            # end = time.time()
            # print("B: ", end - start)

            # merge and update rank
            population = population + offsprings
            population.update_rank()

            # selection
            self.selection(population, [nb_inds_each_task] * len(self.tasks))

            # update operators
            self.crossover.update(population = population)
            self.mutation.update(population = population)

            # save history
            self.history_cost.append([ind.fcost for ind in population.get_solves()])

            #print
            self.render_process((epoch+1)/nb_generations, ['Cost'], [self.history_cost[-1]], use_sys= True)
        
        print('\nEND!')

        #solve 
        self.last_pop = population
        return self.last_pop.get_solves() 


In [39]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(10)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()
# tasks, IndClass = WCCI22_benchmark.get_50tasks_benchmark(10)

In [40]:
baseModel = model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection()
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.5, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Seed: None -- Time: 01m 44.52s  100 % [====================>]  Cost: 3.32E-06  7.84E-07  9.16E-07  0.00E+00  1.27E+02  2.00E+01  1.72E-03  2.25E+03  1.74E-08  2.00E+01  ,  
END!


In [24]:
print(baseModel.E_o)

[  1500.   2000.    800.  59900. 100000.  68200. 100000.  47300.   1200.
   1500.]


In [25]:
print(baseModel.E_s)

[98500. 98000. 99200. 40100.     0. 31800.     0. 52700. 98800. 98500.]


In [26]:
print(baseModel.R_o)

[1.000000e-02 1.000000e-02 1.000000e-02 1.179201e+04 9.784010e+03
 2.509101e+04 3.884301e+04 4.658010e+03 1.000000e-02 1.000000e-02]


In [27]:
print(baseModel.R_s)

[ 922.  933. 1013.  709.    0.  415.    0.  459.  964.  727.]
